In [17]:
import numpy as np
import pandas as pd
import pickle, os, math
from keras.models import Sequential
import tensorflow as tf
from sklearn.model_selection import train_test_split

Import the data first

In [29]:
data_dir = os.path.join(os.curdir, 'Data', 'basic_data.pkl')
with open(data_dir, 'rb') as in_file:
    ot = pickle.load(in_file)
data = ot['data']
target = ot['target']

In [30]:
data.shape

(189418, 164)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.1, random_state=152)

In [32]:
from keras.layers import Dense

In [33]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=164))
model.add(Dense(units=32, activation='relu', input_dim=64))
model.add(Dense(units=1, activation='softmax'))
model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])

In [34]:
model.fit(X_train, y_train, epochs=5, batch_size=128)

Epoch 1/5
170476/170476 [==============================] - 3s 18us/step - loss: 0.0154 - acc: 0.9846
Epoch 2/5
170476/170476 [==============================] - 3s 16us/step - loss: 0.0154 - acc: 0.9846
Epoch 3/5
170476/170476 [==============================] - 3s 16us/step - loss: 0.0154 - acc: 0.9846
Epoch 4/5
170476/170476 [==============================] - 3s 16us/step - loss: 0.0154 - acc: 0.9846
Epoch 5/5
170476/170476 [==============================] - 3s 16us/step - loss: 0.0154 - acc: 0.9846


In [35]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=128)

18942/18942 [==============================] - 0s 16us/step


In [36]:
loss_and_metrics

[0.015626649772991236, 0.9843733502270088]